<a href="https://colab.research.google.com/github/Brackly/Predicting-a-customers-next-purchase-with-Graph-neural-networks/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TASK 1: PREDICTING WHAT A USER WILL BUY

####Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import Tensor
print(torch.__version__)

2.0.1+cu118


In [ ]:
!pip install sentence-transformers
!pip install torch_geometric
import os
os.environ['TORCH'] = torch.__version__
# !pip install torch-sparse
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 125.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=4e4feb5373ee470523c159c098558491f38e0b83c0b559eb8a588b7c3c2da9f5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

####Fetch data

In [ ]:
urls= {
    "products":"https://raw.githubusercontent.com/Brackly/Retail_sales-data-exploration-using-sql/master/SalesLT.Product.csv",
    "sales_orders":"https://raw.githubusercontent.com/Brackly/Retail_sales-data-exploration-using-sql/master/SalesLT.SalesOrderDetail.csv",
    "customer_orders":"https://raw.githubusercontent.com/Brackly/Retail_sales-data-exploration-using-sql/master/SalesLT.SalesOrderHeader.csv"
}

In [ ]:
for url in urls:
  if url == "products":
    products = pd.read_csv(urls[url],index_col="ProductID")
  elif url == "sales_orders":
    sales_orders = pd.read_csv(urls[url],index_col="SalesOrderID")
  else:
    customer_orders = pd.read_csv(urls[url],index_col="SalesOrderID")

####Drop unwanted columns

In [ ]:
columns = ['Color',	'StandardCost',	'ListPrice','Size','Weight','ProductCategoryID','ProductModelID']
for column in columns:
  freq = products[column].dropna().value_counts()
  total_non_nan = products[column].count()
  weights = freq * total_non_nan / freq.sum()
  products[column] = products[column].fillna(pd.Series(np.random.choice(weights.index, size=len(products[column])), index=products.index, name=column))

In [ ]:
products[products['Color'].isna()].count()

Name                      0
ProductNumber             0
Color                     0
StandardCost              0
ListPrice                 0
Size                      0
Weight                    0
ProductCategoryID         0
ProductModelID            0
SellStartDate             0
SellEndDate               0
DiscontinuedDate          0
ThumbNailPhoto            0
ThumbnailPhotoFileName    0
rowguid                   0
ModifiedDate              0
dtype: int64

In [ ]:
import torch

In [ ]:
def drop_columns(df,cols):
  return df.drop(columns=cols)


In [ ]:
Product_cols = ['ProductNumber','SellStartDate','SellEndDate','ThumbNailPhoto','StandardCost','ThumbnailPhotoFileName','Size', 'rowguid', 'ModifiedDate','DiscontinuedDate','ProductModelID','ProductCategoryID']
cleaned_products=drop_columns(products,Product_cols)

In [ ]:
sales_orders_cols = ['rowguid','ModifiedDate']
cleaned_sales_orders=drop_columns(sales_orders,sales_orders_cols)

In [ ]:
customer_orders_cols = ['RevisionNumber','OrderDate','DueDate','ShipDate','Status',
                        'OnlineOrderFlag','SalesOrderNumber','PurchaseOrderNumber','BillToAddressID','ShipMethod','CreditCardApprovalCode',
                        'TaxAmt','Freight','Comment','rowguid','ModifiedDate','ShipToAddressID','AccountNumber','TotalDue']
cleaned_customer_orders=drop_columns(customer_orders,customer_orders_cols)

In [ ]:
# cleaned_products[cleaned_products['Size'].isna()]

In [ ]:
# cleaned_products.ProductCategoryID = cleaned_products.ProductCategoryID.astype(object)

In [ ]:
cleaned_products.dtypes

Name          object
Color         object
ListPrice    float64
Weight       float64
dtype: object

In [ ]:
cleaned_products.head()

,Name,Color,ListPrice,Weight
ProductID,,,,
680,"HL Road Frame - Black, 58",Black,1431.50,1016.04
706,"HL Road Frame - Red, 58",Red,1431.50,1016.04
707,"Sport-100 Helmet, Red",Red,34.99,13008.96
708,"Sport-100 Helmet, Black",Black,34.99,223.00
709,"Mountain Bike Socks, M",White,9.50,6803.85


In [ ]:
cleaned_products.isna().sum()

Name         0
Color        0
ListPrice    0
Weight       0
dtype: int64

In [ ]:
cleaned_products.head()

,Name,Color,ListPrice,Weight
ProductID,,,,
680,"HL Road Frame - Black, 58",Black,1431.50,1016.04
706,"HL Road Frame - Red, 58",Red,1431.50,1016.04
707,"Sport-100 Helmet, Red",Red,34.99,13008.96
708,"Sport-100 Helmet, Black",Black,34.99,223.00
709,"Mountain Bike Socks, M",White,9.50,6803.85


In [ ]:
cleaned_customer_orders.head()

,CustomerID,SubTotal
SalesOrderID,,
71774,29847,880.3484
71776,30072,78.8100
71780,30113,38418.6895
71782,29485,39785.3304
71783,29957,83858.4261


In [ ]:
!pip freeze requirements.txt

absl-py==1.4.0
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
anyio==3.6.2
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
array-record==0.2.0
arviz==0.15.1
astropy==5.2.2
astunparse==1.6.3
attrs==23.1.0
audioread==3.0.0
autograd==1.5
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blis==0.7.9
blosc2==2.0.0
bokeh==2.4.3
branca==0.6.0
build==0.10.0
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.3.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.3
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.4
cons==0.4.5
contextlib2==0.6.0.post1
contourpy==1.0.7
convertdate==2.4.0
cryptography==40.0.2
cufflinks==0.17.3
cupy-cuda11x==11.0.0
cvxopt==1.3.0
cvxpy==1.3.1
cycler==0.11.0
cymem==2.0.7
Cython==0.29.34
dask==2022.12.1
datascience==0.17.6
db-dtypes==1.1.1
dbus-python==1.2.16
debugpy==1.6.6
decorator==4.4.2

In [ ]:
cleaned_sales_orders.head()

,SalesOrderDetailID,OrderQty,ProductID,UnitPrice,UnitPriceDiscount,LineTotal
SalesOrderID,,,,,,
71774,110562,1,836,356.898,0.0,356.898
71774,110563,1,822,356.898,0.0,356.898
71776,110567,1,907,63.900,0.0,63.900
71780,110616,4,905,218.454,0.0,873.816
71780,110617,2,983,461.694,0.0,923.388


In [ ]:
# test = cleaned_products.drop(columns=['StandardCost',	'ListPrice','Size','Weight','ProductCategoryID','ProductModelID'])

In [ ]:
orders = pd.merge(cleaned_sales_orders, cleaned_customer_orders, on='SalesOrderID')

In [ ]:
orders.head()

,SalesOrderDetailID,OrderQty,ProductID,UnitPrice,UnitPriceDiscount,LineTotal,CustomerID,SubTotal
SalesOrderID,,,,,,,,
71774,110562,1,836,356.898,0.0,356.898,29847,880.3484
71774,110563,1,822,356.898,0.0,356.898,29847,880.3484
71776,110567,1,907,63.900,0.0,63.900,30072,78.8100
71780,110616,4,905,218.454,0.0,873.816,30113,38418.6895
71780,110617,2,983,461.694,0.0,923.388,30113,38418.6895


In [ ]:
orders2=orders.drop(columns=['SalesOrderDetailID','ProductID'])

In [ ]:
customers = orders2.groupby('CustomerID').mean()

In [ ]:
customers.head()

,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,SubTotal
CustomerID,,,,,
29485,3.255814,415.816465,0.000000,774.883395,39785.3304
29531,3.833333,400.014033,0.003333,922.311485,6634.2961
29546,3.976190,410.747429,0.000000,1765.719714,88812.8625
29568,1.375000,210.073500,0.050000,232.025850,2415.6727
29584,2.333333,30.526000,0.000000,73.752000,246.7392


In [ ]:
orders[orders['CustomerID']==761]

,SalesOrderDetailID,OrderQty,ProductID,UnitPrice,UnitPriceDiscount,LineTotal,CustomerID,SubTotal
SalesOrderID,,,,,,,,


In [ ]:
products_path='products.csv'
customer_path='customer.csv'
orders_path = 'orders.csv'
cleaned_products.to_csv(products_path)
orders.to_csv(orders_path)
customers.to_csv(customer_path)

In [ ]:
cleaned_products.shape

(295, 4)

In [ ]:
!ls -al

total 68
drwxr-xr-x 1 root root  4096 Jun  2 09:00 .
drwxr-xr-x 1 root root  4096 Jun  2 08:57 ..
drwxr-xr-x 4 root root  4096 May 31 13:45 .config
-rw-r--r-- 1 root root  2158 Jun  2 09:00 customer.csv
-rw-r--r-- 1 root root 29949 Jun  2 09:00 orders.csv
-rw-r--r-- 1 root root 14434 Jun  2 09:00 products.csv
drwxr-xr-x 1 root root  4096 May 31 13:46 sample_data


##Graph

In [ ]:
import torch

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA device assigned:', device)
else:
    device = 'cpu'
    print('CUDA is not available')

CUDA device assigned: cuda


In [ ]:
from sentence_transformers import SentenceTransformer
class SequenceEncoder(object):
    def __init__(self, model_name='all-MiniLM-L6-v2', device=device):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

In [ ]:
class GenresEncoder(object):
    def __init__(self, sep='|'):
        self.sep = sep

    def __call__(self, df):
        genres = set(g for col in df.values for g in col.split(self.sep))
        mapping = {genre: i for i, genre in enumerate(genres)}

        x = torch.zeros(len(df), len(mapping))
        for i, col in enumerate(df.values):
            for genre in col.split(self.sep):
                x[i, mapping[genre]] = 1
        return x

In [ ]:
class NumericalEncoder(object):
    def __call__(self, df):
        x = torch.from_numpy(df.values)
        return x

In [ ]:
from sklearn.preprocessing import MinMaxScaler
class ScalerEncoder(object):
    def __init__(self, feature_range=(0, 1)):
        self.feature_range = feature_range
        self.scaler = MinMaxScaler(feature_range=feature_range)

    def __call__(self, df):
        x = self.scaler.fit_transform(df.values.reshape(-1, 1))
        return torch.tensor(x, dtype=torch.float32)

In [ ]:
def load_node_csv(path, index_col, encoders=None,**kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    encoded= None
    if encoders is not None:
      xs = [encoder(df[col]) for col, encoder in encoders.items()]
      x = torch.cat(xs, dim=-1)
    return x, mapping

In [ ]:
product_x, product_mapping = load_node_csv(
    products_path, index_col='ProductID', encoders={
        'Name': SequenceEncoder(),
        'Color': GenresEncoder(),
        'ListPrice': ScalerEncoder(),
        'Weight': ScalerEncoder()
    })

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
product_x.shape

torch.Size([295, 395])

In [ ]:
user, user_mapping = load_node_csv(customer_path, index_col='CustomerID', encoders={
    'OrderQty':ScalerEncoder(),
    'UnitPrice':ScalerEncoder(),
    'UnitPriceDiscount':ScalerEncoder(),
    'LineTotal':ScalerEncoder(),
    'SubTotal':ScalerEncoder(),
})

In [ ]:
user_mapping

{29485: 0,
 29531: 1,
 29546: 2,
 29568: 3,
 29584: 4,
 29612: 5,
 29638: 6,
 29644: 7,
 29653: 8,
 29660: 9,
 29736: 10,
 29741: 11,
 29781: 12,
 29796: 13,
 29847: 14,
 29877: 15,
 29922: 16,
 29929: 17,
 29932: 18,
 29938: 19,
 29957: 20,
 29975: 21,
 29982: 22,
 30019: 23,
 30025: 24,
 30027: 25,
 30033: 26,
 30050: 27,
 30072: 28,
 30089: 29,
 30102: 30,
 30113: 31}

In [ ]:
user.shape

torch.Size([32, 5])

##Loading data to the Graph

In [ ]:
from torch_geometric.data import HeteroData

data = HeteroData()

data['customer'].x = user
data['product'].x = product_x
data["customer"].node_id = torch.arange(len(user))
data["product"].node_id = torch.arange(len(product_x))

In [ ]:
data2 = HeteroData()
data2["customer"].x = user
data2["customer"].node_id = torch.arange(len(user))
# Since the new customers have not bought any products, you can leave the product node features and edge indices empty.
# For example:
data2["product"].x = torch.Tensor([])
data["product"].node_id = torch.arange(len(product_x))  # Empty tensor
data2.edge_index = []

In [ ]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    df = pd.read_csv(path, **kwargs)

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

In [ ]:
class IdentityEncoder(object):
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [ ]:
import torch_geometric.transforms as T

edge_index, edge_label = load_edge_csv(
    orders_path,
    src_index_col='CustomerID',
    src_mapping=user_mapping,
    dst_index_col='ProductID',
    dst_mapping=product_mapping,
    encoders={'OrderQty': IdentityEncoder(dtype=torch.long)},
)

# Set the edge attributes for the 'buys' and 'bought_by' edges
data['customer', 'buys', 'product'].edge_index = edge_index
data['customer', 'buys', 'product'].edge_label = edge_label
data = T.ToUndirected()(data)


In [ ]:
data

HeteroData(
  customer={
    x=[32, 5],
    node_id=[32],
  },
  product={
    x=[295, 395],
    node_id=[295],
  },
  (customer, buys, product)={
    edge_index=[2, 542],
    edge_label=[542, 1],
  },
  (product, rev_buys, customer)={
    edge_index=[2, 542],
    edge_label=[542, 1],
  }
)

##Splitting data

In [ ]:
import torch_geometric.transforms as T
transform = T.RandomLinkSplit(
    num_val=0.1,  # TODO
    num_test=0.1,  # TODO
    disjoint_train_ratio=0.3,  # TODO
    neg_sampling_ratio=2.0,  # TODO
    add_negative_train_samples=False,  # TODOg
    edge_types=("customer", "buys", "product"),
    rev_edge_types=("product", "rev_buys", "customer"),
)

train_data, val_data, test_data = transform(data)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)


Training data:
HeteroData(
  customer={
    x=[32, 5],
    node_id=[32],
  },
  product={
    x=[295, 395],
    node_id=[295],
  },
  (customer, buys, product)={
    edge_index=[2, 304],
    edge_label=[130, 1],
    edge_label_index=[2, 130],
  },
  (product, rev_buys, customer)={
    edge_index=[2, 304],
    edge_label=[304, 1],
  }
)

Validation data:
HeteroData(
  customer={
    x=[32, 5],
    node_id=[32],
  },
  product={
    x=[295, 395],
    node_id=[295],
  },
  (customer, buys, product)={
    edge_index=[2, 434],
    edge_label=[162, 1],
    edge_label_index=[2, 162],
  },
  (product, rev_buys, customer)={
    edge_index=[2, 434],
    edge_label=[434, 1],
  }
)


###Batching data

In [ ]:
from torch_geometric.loader import LinkNeighborLoader

# Define seed edges:
edge_label_index = train_data["customer", "buys", "product"].edge_label_index
edge_label = train_data["customer", "buys", "product"].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,  # TODO
    num_neighbors=[20,10],  # TODO
    neg_sampling_ratio=2,  # TODO
    edge_label_index=(("customer", "buys", "product"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  customer={
    x=[32, 5],
    node_id=[32],
    n_id=[32],
  },
  product={
    x=[224, 395],
    node_id=[224],
    n_id=[224],
  },
  (customer, buys, product)={
    edge_index=[2, 304],
    edge_label=[384, 1],
    edge_label_index=[2, 384],
    e_id=[304],
    input_id=[128],
  },
  (product, rev_buys, customer)={
    edge_index=[2, 269],
    edge_label=[269, 1],
    e_id=[269],
  }
)


In [ ]:
data

HeteroData(
  customer={
    x=[32, 5],
    node_id=[32],
  },
  product={
    x=[295, 395],
    node_id=[295],
  },
  (customer, buys, product)={
    edge_index=[2, 542],
    edge_label=[542, 1],
  },
  (product, rev_buys, customer)={
    edge_index=[2, 542],
    edge_label=[542, 1],
  }
)

In [ ]:
data['product'].x[0].shape

torch.Size([395])

##Graph Neural Net

In [ ]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x
# The final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_customer: Tensor, x_product: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_customer = x_customer[edge_label_index[0]]
        edge_feat_product = x_product[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_customer * edge_feat_product).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for customers and products:
        self.product_lin = torch.nn.Linear(data["product"].x.size(1), hidden_channels)
        self.customer_lin = torch.nn.Linear(data["customer"].x.size(1), hidden_channels)
        self.customer_emb = torch.nn.Embedding(data["customer"].num_nodes, hidden_channels)
        self.product_emb = torch.nn.Embedding(data["product"].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "customer": self.customer_lin(data["customer"].x) + self.product_emb(data["customer"].node_id),
          "product": self.product_lin(data["product"].x) + self.product_emb(data["product"].node_id),
        }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)

        pred = self.classifier(
            x_dict["customer"],
            x_dict["product"],
            data["customer", "buys", "product"].edge_label_index,
        )

        return pred


model = Model(hidden_channels=32)

print(model)

Model(
  (product_lin): Linear(in_features=395, out_features=32, bias=True)
  (customer_lin): Linear(in_features=5, out_features=32, bias=True)
  (customer_emb): Embedding(32, 32)
  (product_emb): Embedding(295, 32)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (customer__buys__product): SAGEConv(32, 32, aggr=mean)
      (product__rev_buys__customer): SAGEConv(32, 32, aggr=mean)
    )
    (conv2): ModuleDict(
      (customer__buys__product): SAGEConv(32, 32, aggr=mean)
      (product__rev_buys__customer): SAGEConv(32, 32, aggr=mean)
    )
  )
  (classifier): Classifier()
)


##TRAINING

In [ ]:
import tqdm
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 7):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()

        # TODO: Move `sampled_data` to the respective `device`
        sampled_data = sampled_data.to(device)
        # TODO: Run `forward` pass of the model
        pred=model.forward(sampled_data)
        # TODO: Apply binary cross entropy via
        # `F.binary_cross_entropy_with_logits(pred, ground_truth)`
        loss = F.binary_cross_entropy_with_logits(pred, sampled_data["customer", "buys", "product"]['edge_label'].squeeze().float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cuda'


100%|██████████| 2/2 [00:00<00:00, 61.40it/s]


Epoch: 001, Loss: 0.4618


100%|██████████| 2/2 [00:00<00:00, 100.60it/s]


Epoch: 002, Loss: -0.2924


...

Epoch: 004, Loss: -1.5992


100%|██████████| 2/2 [00:00<00:00, 91.08it/s]


Epoch: 005, Loss: -2.2447


100%|██████████| 2/2 [00:00<00:00, 104.76it/s]

Epoch: 006, Loss: -3.0605


## Model Prediction

In [ ]:
edge_label_index = val_data["customer", "buys", "product"].edge_label_index
edge_label = val_data["customer", "buys", "product"].edge_label

val_loader = LinkNeighborLoader(
    data=data,  # TODO
    num_neighbors=[20,10],  # TODO
    neg_sampling_ratio=2,  # TODO
    edge_label_index=(("customer", "buys", "product"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  customer={
    x=[32, 5],
    node_id=[32],
    n_id=[32],
  },
  product={
    x=[241, 395],
    node_id=[241],
    n_id=[241],
  },
  (customer, buys, product)={
    edge_index=[2, 538],
    edge_label=[384, 1],
    e_id=[538],
    input_id=[128],
    edge_label_index=[2, 384],
  },
  (product, rev_buys, customer)={
    edge_index=[2, 331],
    edge_label=[331, 1],
    e_id=[331],
  }
)


In [ ]:
def pred_mapping(item_dict:dict) -> dict:
  pred_mapping = {}
  for value in item_dict:
    pred_mapping[item_dict[value]] = value
  return pred_mapping

In [ ]:
data['customer'].node_id

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [ ]:
customer_pred_mapping = pred_mapping(user_mapping)
product_pred_mapping = pred_mapping(product_mapping)
preds = []
ground_truths = []
customer_ids = []
product_ids = []
for sampled_data in val_loader:
    with torch.no_grad():
        sampled_data = sampled_data.to(device)
        pred=model.forward(sampled_data)
        ground_truth = sampled_data["customer", "buys", "product"]['edge_label'].squeeze().float()
        preds.append(pred)
        customer_ids.append(sampled_data['customer']['node_id'])
        product_ids.append(sampled_data['product']['node_id'])
        ground_truths.append(ground_truth)

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
customer_ids = torch.cat(customer_ids, dim=0).cpu().numpy()
product_ids = torch.cat(product_ids, dim=0).cpu().numpy()


In [ ]:
res=[]
productid=761
for i in range(1,50):
  # if product_pred_mapping[product_ids[i]] == int(productid):
    # print(f' \n-------  Prediction {i} ------\n')
    responseobj = {}
    responseobj['Customer ID']= customer_pred_mapping[customer_ids[i]]
    responseobj['Product ID']=product_pred_mapping[product_ids[i]]
    responseobj['Predicted Quantity']=pred[i]
    responseobj['ground_truth']=ground_truth[i]
    res.append(responseobj)
    # print('-> True value : ',ground_truth[i])
    # print('---------------------------------------\n')
res

[{'Customer ID': 29531,
  'Product ID': 707,
  'Predicted Quantity': 5.037572,
  'ground_truth': 4.0},
 {'Customer ID': 29546,
  'Product ID': 708,
  'Predicted Quantity': 5.052694,
  'ground_truth': 3.0},
 {'Customer ID': 29568,
  'Product ID': 710,
  'Predicted Quantity': 2.3953376,
  'ground_truth': 1.0},
 {'Customer ID': 29584,
  'Product ID': 711,
  'Predicted Quantity': 4.0102386,
  'ground_truth': 5.0},
 {'Customer ID': 29612,
  'Product ID': 712,
  'Predicted Quantity': 1.7122726,
  'ground_truth': 1.0},
 {'Customer ID': 29638,
  'Product ID': 715,
  'Predicted Quantity': 2.962041,
  'ground_truth': 3.0},
 {'Customer ID': 29644,
  'Product ID': 717,
  'Predicted Quantity': 3.4103813,
  'ground_truth': 1.0},
 {'Customer ID': 29653,
  'Product ID': 718,
  'Predicted Quantity': 1.3427861,
  'ground_truth': 1.0},
 {'Customer ID': 29660,
  'Product ID': 719,
  'Predicted Quantity': 3.3525908,
  'ground_truth': 5.0},
 {'Customer ID': 29736,
  'Product ID': 720,
  'Predicted Quantity'

In [ ]:
preds[0]['predictedQuantity'].shape

IndexError: ignored

In [ ]:
data

In [ ]:
print(model.state_dict())

##Metrics

In [ ]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(ground_truth, pred,multi_class='ovr')
print()
print(f"Validation AUC: {auc:.4f}")

AxisError: ignored